<a href="https://colab.research.google.com/github/kirthi1313/repo2/blob/main/Final_SciBERT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash
DATADIR="NCBI_disease"
if test ! -d "$DATADIR";then
    echo "Creating $DATADIR dir"
    mkdir "$DATADIR"
    cd "$DATADIR"
    wget https://raw.githubusercontent.com/kirthi1313/repo2/main/sample.txt
    wget https://raw.githubusercontent.com/kirthi1313/repo2/main/sample1.txt
    wget https://raw.githubusercontent.com/kirthi1313/repo2/main/sample1.txt
fi

In [3]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import statistics as stats

!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

#sys.path.append("../") 
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
     |████████████████████████████████| 133kB 6.7MB/s 
     |████████████████████████████████| 7.6MB 10.7MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp37-none-any.whl size=54248 sha256=57aa798dcd9dd50887ae24c01d1c3afc04c3db80f3b16b1e4909674573fd9a18
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
ERROR: botocore 1.20.86 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [4]:
def read_tsv(filename, quotechar=None):
    with open(filename, "r", encoding='utf-8') as f:
        return list(csv.reader(f, delimiter="\t", quotechar=quotechar))   


In [5]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [6]:
def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""
    
    # read file
    lines =  open(filename).read().strip()  

    
    # find sentence-like boundaries
    lines = lines.split("\n\n")  
    
     # split on newlines
    lines = [line.split("\n") for line in lines]
    
    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]
    print(lines)
    # get labels/tags
    # labels = []
    # for line in lines:
    #   for l in line:
    #     if(l.split()[idx]):
    #       labels.append(l.split()[idx])
    labels = [[l.split()[3] for l in line] for line in lines]
    
    #convert to df
    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)
    return df


In [7]:
DATADIR = "NCBI_disease/"

def get_data(trainfile=DATADIR + "sample.txt",
             devfile=DATADIR + "sample1.txt",
             testfile=DATADIR + "sample1.txt.1"):

    train = read_CoNLL2003_format(trainfile, idx=3)    
    dev = read_CoNLL2003_format(devfile, idx=3)
    
    # combine train and dev
    train = pd.concat([train, dev])
    print("Train and dev data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    test = read_CoNLL2003_format(testfile, idx=3)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))
    
    return train, test

In [8]:
train, test = get_data()
print(test)

X_train, y_train = train.tokens, train.labels
X_test, y_test = test.tokens, test.labels

print(len(train))

label_list = np.unique(flatten(y_train))
label_list = list(label_list)
print("\nNER tags:",label_list)
print(X_test,y_test)

[['Stay NN O O', 'active NN O CHAT', 'and NN O O', 'lets NN O O', 'have NN O O', 'a NN O O', 'chat NN O CHAT', '. NN O O'], ['Chatbot NN O CHAT', 'has NN O O', 'answered NN O CHAT', '. NN O O'], ['Communicate NN O CHAT', 'with NN O O', 'the NN O O', 'users NN O O', '. NN O O'], ['We NN O O', 'received NN O CHAT', 'a NN O O', 'conversation NN O CHAT', '. NN O O'], ['We NN O O', 'chatted NN O CHAT', 'with NN O O', 'a NN O O', 'robot NN O CHAT', '. NN O O'], ['Is NN O O', 'it NN O O', 'an NN O O', 'automated NN O CHAT', 'message NN O CHAT', '? NN O O'], ['Tell NN O CHAT', 'me NN O O', 'about NN O O', 'yourself NN O O', '? NN O O'], ['I NN O O', 'need NN O O', 'to NN O O', 'know NN O CHAT', 'more NN O CHAT', 'about NN O CHAT', 'you NN O O', '. NN O O'], ['I NN O O', 'never NN O O', 'spoke NN O CHAT', 'with NN O O', 'an NN O O', 'AI NN O CHAT', 'before NN O O', '. NN O O'], ['Can NN O O', 'you NN O O', 'answer NN O CHAT', 'please NN O O', '? NN O O']]
[['Did NN O O', 'you NN O O', 'get NN O

In [9]:
print(train)

                                               tokens                                      labels
0         [Stay, active, and, lets, have, a, chat, .]              [O, CHAT, O, O, O, O, CHAT, O]
1                         [Chatbot, has, answered, .]                          [CHAT, O, CHAT, O]
2                  [Communicate, with, the, users, .]                          [CHAT, O, O, O, O]
3                  [We, received, a, conversation, .]                       [O, CHAT, O, CHAT, O]
4                    [We, chatted, with, a, robot, .]                    [O, CHAT, O, O, CHAT, O]
5                 [Is, it, an, automated, message, ?]                    [O, O, O, CHAT, CHAT, O]
6                      [Tell, me, about, yourself, ?]                          [CHAT, O, O, O, O]
7            [I, need, to, know, more, about, you, .]           [O, O, O, CHAT, CHAT, CHAT, O, O]
8          [I, never, spoke, with, an, AI, before, .]              [O, O, CHAT, O, O, CHAT, O, O]
9                   

In [10]:
i = 3
tokens = X_test[i]
labels = y_test[i]

data = {"token": tokens,"label": labels}
df=pd.DataFrame(data=data)
print(df)

       token label
0        Can     O
1         we     O
2       talk  CHAT
3      about     O
4  something     O
5       that     O
6  interests  CHAT
7         me     O
8          ?     O


In [20]:
%%time
model = BertTokenClassifier(bert_model='bert-base-cased',
                            max_seq_length=178,
                            do_lower_case=True,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=6,
                            eval_batch_size=6,
                            validation_fraction=0., 
                            label_list=None,                           
                            ignore_label=['O'])

print(model)

Building sklearn token classifier...
BertTokenClassifier(bert_config_json=None, bert_model='bert-base-cased',
                    bert_vocab=None, do_lower_case=True, epochs=3,
                    eval_batch_size=6, fp16=False, from_tf=False,
                    gradient_accumulation_steps=4, ignore_label=['O'],
                    label_list=None, learning_rate=3e-05, local_rank=-1,
                    logfile='bert_sklearn.log', loss_scale=0,
                    max_seq_length=178, num_mlp_hiddens=500, num_mlp_layers=0,
                    random_state=42, restore_file=None, train_batch_size=6,
                    use_cuda=True, validation_fraction=0.0,
                    warmup_proportion=0.1)
CPU times: user 3.01 ms, sys: 123 µs, total: 3.13 ms
Wall time: 3.19 ms


In [21]:
# finetune model on train data
model.fit(X_train, y_train)


Loading bert-base-cased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 22, validation data size: 0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


BertTokenClassifier(bert_config_json=None, bert_model='bert-base-cased',
                    bert_vocab=None, do_lower_case=True, epochs=3,
                    eval_batch_size=6, fp16=False, from_tf=False,
                    gradient_accumulation_steps=4, ignore_label=['O'],
                    label_list=array(['CHAT', 'O'], dtype='<U4'),
                    learning_rate=3e-05, local_rank=-1,
                    logfile='bert_sklearn.log', loss_scale=0,
                    max_seq_length=178, num_mlp_hiddens=500, num_mlp_layers=0,
                    random_state=42, restore_file=None, train_batch_size=6,
                    use_cuda=True, validation_fraction=0.0,
                    warmup_proportion=0.1)

In [22]:
# get predictions on test data
y_preds = model.predict(X_test)
print(X_test,y_test,y_preds)

# print report on classifier stats
print(classification_report(flatten(y_test), flatten(y_preds)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



0                          [Did, you, get, my, text, ?]
1                           [I, received, the, text, .]
2                     [When, shall, I, message, you, ?]
3     [Can, we, talk, about, something, that, intere...
4     [Are, you, busy?, I, want, to, talk, to, someo...
5                    [We, received, a, conversation, .]
6                      [We, chatted, with, a, robot, .]
7                   [Is, it, an, automated, message, ?]
8                        [Tell, me, about, yourself, ?]
9              [I, need, to, know, more, about, you, .]
10           [I, never, spoke, with, an, AI, before, .]
11                        [Can, you, answer, please, ?]
Name: tokens, dtype: object 0                          [O, O, O, O, CHAT, O]
1                             [O, O, O, CHAT, O]
2                          [O, O, O, CHAT, O, O]
3              [O, O, CHAT, O, O, O, CHAT, O, O]
4     [O, O, CHAT, O, O, O, CHAT, O, O, O, O, O]
5                          [O, CHAT, O, CHAT, O]
6    